In [0]:
dbutils.widgets.text("english_documents", "/FileStore/tables/cfuentes/english_articles.parquet")
dbutils.widgets.text("origin_path", "/FileStore/tables/cfuentes/input/{}.json")
dbutils.widgets.text("error_destination_path", "/FileStore/tables/cfuentes/error/{}.json")
dbutils.widgets.text("processed_destination_path", "/FileStore/tables/cfuentes/processed/{}.json")

In [0]:
parquet_path = dbutils.widgets.get("english_documents")
origin_path = dbutils.widgets.get("origin_path")
error_destination_path = dbutils.widgets.get("error_destination_path")
processed_destination_path = dbutils.widgets.get("processed_destination_path")

In [0]:
parquetDF = spark.read.parquet(parquet_path)

parquetDF.show()

+--------------------+--------------------+--------+--------------------+-------------+----------------+
 paper_id| abstract_text|language| authors|es_has_errors|graph_has_errors|
+--------------------+--------------------+--------+--------------------+-------------+----------------+
002d9f25d86a561f5...|The study of the ...| en|[Łukasz Krawiec, ...| false| false|
002c8c43cea5f3ec8...|Microbial life pe...| en|[Marjorie A Chan,...| false| false|
002ac207ff68ec921...|We conducted pros...| en|[P G Szilagyi, A ...| false| false|
0028a11473d831195...| | en| []| true| true|
00286c2b94f42979d...|5 Objective: To a...| en|[Hubert Blain, Yv...| false| false|
0025aba7979aa2756...| | en| []| true| true|
0000028b5cc154f68...| | en| []| true| true|
002b48e6f0350e5ce...| | en| []| true| true|
+--------------------+--------------------+--------+--------------------+-------------+----------------+

In [0]:
from pyspark.sql.functions import col

error_list = [r[0] for r in parquetDF.filter(col("es_has_errors") | col("graph_has_errors")).select('paper_id').toLocalIterator()]
print(error_list)
processed_list = [r[0] for r in parquetDF.filter((~col("es_has_errors")) & (~col("graph_has_errors"))).select('paper_id').toLocalIterator()]
print(processed_list)

['0028a11473d831195af7797de9c9985b65717bfd', '0025aba7979aa27567a65cf6f19d6120f7f4691b', '0000028b5cc154f68b8a269f6578f21e31f62977', '002b48e6f0350e5ce48a1cf54e62b876f286a800']
['002d9f25d86a561f56d568e606c26c7b5e922f35', '002c8c43cea5f3ec8c97d30fe181c01b919d789a', '002ac207ff68ec92179d4510a1623a9cc6c6acec', '00286c2b94f42979d234bd4070fba1d82006e61b']

In [0]:
for error_id in error_list:
  dbutils.fs.mv(origin_path.format(error_id), error_destination_path.format(error_id))

In [0]:
for processed_id in processed_list:
  dbutils.fs.mv(origin_path.format(processed_id), processed_destination_path.format(processed_id))

In [0]:
dbutils.notebook.exit("Ok")